In [47]:
from bs4 import BeautifulSoup

In [66]:
from selenium import webdriver
from time import sleep
import pandas
from tqdm import tqdm_notebook

In [ ]:
dt_index = pandas.date_range(start='20190601',end=pandas.Timestamp.today())
#dt_index = pandas.date_range(start='20190107',end='20190107')
dt_list = dt_index.strftime("%Y.%m.%d").tolist()

bar_total = tqdm_notebook(dt_list)
market_date = []
market_company = [] 
market_crop = []
market_crop_spec = []
market_scale = []
market_grade = []
market_price = []
market_origin = []
market_eco = []

try:
    

    for item in bar_total:
        browser = webdriver.Chrome("C:/Users/shin/Downloads/chromedriver_win32/chromedriver.exe")
        js = '''
          form = document.createElement('form');
          form.method = 'post';
          form.action = 'https://www.garak.co.kr/price/OZViewer.do';
          form.acceptCharset="UTF-8";

          hiddenField = document.createElement('input');
          hiddenField.type = 'hidden';
          hiddenField.name = "s_pummok";
          hiddenField.value = "바실";
          form.appendChild(hiddenField);

          hiddenField = document.createElement('input');
          hiddenField.type = 'hidden';
          hiddenField.name = 's_date';
          hiddenField.value = "'''+item+ '''";
          form.appendChild(hiddenField);

          hiddenField = document.createElement('input');
          hiddenField.type = 'hidden';
          hiddenField.name = "R010680";
          hiddenField.value = "10";
          form.appendChild(hiddenField);

          hiddenField = document.createElement('input');
          hiddenField.type = 'hidden';
          hiddenField.name = "R010690";
          hiddenField.value = "10";
          form.appendChild(hiddenField);

          hiddenField = document.createElement('input');
          hiddenField.type = 'hidden';
          hiddenField.name = "R010700";
          hiddenField.value = "10";
          form.appendChild(hiddenField);

          document.body.appendChild(form);
          form.submit();
        '''
        browser.execute_script(js)
        sleep(4)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        list_tspan = soup.find_all('tspan')

        #oz viewer page 수
        page_cnt = int(list_tspan[3].get_text()[2:])

        #페이지 별 출력을 위해 js code 실행
        js_click_event = 'document.getElementsByClassName("btnNEXT")[0].click();'

        for i in range(1,page_cnt+1):
            if i > 1 :
                browser.execute_script(js_click_event)
                sleep(2)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            list_tspan = soup.find_all('tspan')
            list_tspan = list_tspan[13:-7]
            tmp =  len(list_tspan)%8
            if tmp == 0:
                line_cnt = int(len(list_tspan)/8)
                for j in range(0,line_cnt):
                    temp = j*8
                    market_date.append(item)
                    market_company.append(list_tspan[temp].get_text())
                    market_crop.append(list_tspan[temp+1].get_text())
                    market_crop_spec.append(list_tspan[temp+2].get_text())
                    market_scale.append(list_tspan[temp+3].get_text())
                    market_grade.append(list_tspan[temp+4].get_text())
                    market_price.append(list_tspan[temp+5].get_text())
                    market_origin.append(list_tspan[temp+6].get_text())
                    market_eco.append(list_tspan[temp+7].get_text())

        #브라우저를 닫는다.
        browser.close()
except:
    print(item,'에서 에러났땅')
    #data frame 생성
    data = {'일자':market_date, '법인':market_company, '품목':market_crop, '품종':market_crop_spec, '단위':market_scale, '등급':market_grade, '경락':market_price, '출하지':market_origin, '친환경 구분':market_eco}
    df = pandas.DataFrame(data)
    #df.head()
    df.to_csv('basil.csv', sep=',',encoding='UTF-8')
    
#data frame 생성
data = {'일자':market_date, '법인':market_company, '품목':market_crop, '품종':market_crop_spec, '단위':market_scale, '등급':market_grade, '경락':market_price, '출하지':market_origin, '친환경 구분':market_eco}
df = pandas.DataFrame(data)
#df.head()
df.to_csv('basil.csv', sep=',',encoding='UTF-8')